In [ ]:
import requests, json, time, os
from lxml import html
from bs4 import BeautifulSoup
import pandas as pd
import pickle

In [ ]:
# Global Variable
DELAY_SECONDS = 2

## 登入
* 登入後重複用 `session_request` 去爬其他資料

In [ ]:
EMAIL = 'tony71318@yahoo.com.tw'
PASSWORD = 'Bot12345'

LOGIN_URL = 'https://www.bundesliga.com/en/bundesliga/login/'
api_key = '3_tQWqB95PCJiqCeljbI7N3oOEiX_q4-wobViZ6EYqtb6vYsZDYQqgPT1Z4O6btnKh'

In [ ]:
headers = {
    'Cookie': 'gmid=J8dn6yAqHjxSZ8kTmbjmVno6Dn3eBcM2GCcc9wR95jI=; hasGmid=ver2; apiDomain_3_dmGIVZ011qA426N4BdhbPVrAqPj8Ve_z6Y1OIC9wOXz2bvKj37HhXXqcaKsESJIE=eu1.gigya.com; ucid=tIzK1ZQCy7sH8VQXbT7itg',
    'Referer': 'https://cdns.eu1.gigya.com/gs/webSdk/Api.aspx?apiKey=3_tQWqB95PCJiqCeljbI7N3oOEiX_q4-wobViZ6EYqtb6vYsZDYQqgPT1Z4O6btnKh&version=latest',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'
                   'AppleWebKit/537.36 (KHTML, like Gecko)'
                   'Chrome/69.0.3497.92 Safari/537.36',
}

In [ ]:
payload = {
    'loginID': EMAIL,
    'password': PASSWORD,
    'sessionExpiration': '0',
    'targetEnv': 'jssdk',
    'include': 'profile,data,emails,subscriptions,preferences,',
    'includeUserInfo': 'true',
    'loginMode': 'standard',
    'lang': 'en',
    'APIKey': api_key,
    'source': 'showScreenSet',
    'sdk': 'js_latest',
    'authMode': 'cookie',
    'pageURL': 'https://www.bundesliga.com/en/bundesliga/login/',
    'format': 'jsonp',
    'callback': 'gigya.callback',
    'context': 'R1396585807',
    'utf8': '&#x2713'
}

In [ ]:
session_requests = requests.session()
result = session_requests.post(LOGIN_URL, data = payload, headers = headers)
result

---
## 得分排行榜
* 回傳: json格式
    * Metadata
    * Players
    * Rankings
    * Teams
    
**TODO: 網頁表格 是從這個json組出來的 交給你想辦法 (*Not* DONE)**

In [ ]:
headers = {
    'Referer': 'https://www.bundesliga.com/en/stats/bundesliga/current-season/all-matchdays/player-stats/goals',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)' 
    'AppleWebKit/537.36 (KHTML, like Gecko)'
    'Chrome/69.0.3497.92 Safari/537.36',
    'x-api-key': 'HEEyMDwtUkR8FUbSj8bx8jqy'
}

In [ ]:
api_url = 'https://xthokrkm7l.execute-api.eu-central-1.amazonaws.com/Prod/ranking?feedType=Goals&timeScope=Season&dataScope=Competition&competitionId=DFL-COM-000001&seasonId=DFL-SEA-0001K2&matchDayId=DFL-DAY-004BPN&rankingType=GOALS&rankingScope=PLAYER'
url = session_requests.get(api_url, headers = headers) 
score_json = json.loads(url.text)
score_json

In [ ]:
# 將三個資料存成df(暫時不知道可以怎麼用 先不存)
players = pd.DataFrame(score_json['Players'])
rankings = pd.DataFrame(score_json['Ranking'])
teams = pd.DataFrame(score_json['Teams'])

---
## Matchday 1-34 2017/18
[Target Link](https://www.bundesliga.com/en/bundesliga/matchday/2018-2019/)
* 整理成df格式
* pickle檔案為保留型態的檔案(之後分析好用)

** 待解決 **
* 找出有用的columns(不急)

In [ ]:
# 抓matchday1~34 2017/18 抓完delay一會 避免被Ban
matchday = {}
for i in range(34):
    api_url = 'https://www.bundesliga.com/fsSolrSearch/search?wt=json&rows=150&sort=kickoffTimeMS_l%20asc&q=indexName%3ASyncMatches%20AND%20competitionID_tlc%3ADFL-COM-000001%20AND%20seasonID_tlc%3A2017%2F2018%20AND%20matchday_l%3A'+str(i+1)
    url = session_requests.get(api_url, headers = headers) 
    matchday[i] = json.loads(url.text)
    time.sleep(DELAY_SECONDS)

In [ ]:
df_matchday = pd.DataFrame(matchday[0]['response']['docs'])
for i in range(1,34):
    df_matchday = pd.concat([df_matchday,pd.DataFrame(matchday[i]['response']['docs'])])

In [ ]:
# 把資料存起來
with open('Data/matchday.pickle', 'wb') as handle:
    pickle.dump(df_matchday, handle, protocol=pickle.HIGHEST_PROTOCOL)

### 開啟方式如下
```
with open('Data/matchday.pickle', 'rb') as handle:
    df_matchday = pickle.load(handle)
```

---

## TODO
* 希望可以用程式直接連button作後續的處理
> Jac: 第一個就是希望 可以更自動一點 像我原本的程式裡的那個網址最後是+str()因為有找到規律 但是不知道有沒有方法讓他去抓matchday1的那個下拉選單去改
以後也可能要做 點進每場比賽的動作 總不能34天每場都自己點進去再抓ㄅ
* 連結單場比賽獲得單場各項數據(連不到可以用的html)
> Jac: 第二個 就是 match day 不是會有一堆比賽 你可以點進去任何一場看細節 在match stats裡面有很多讚讚的資料 但是我抓不下來

---
# Matchday statistics
    從Matchday獲得當日比賽的資訊
    從中抓取每場比賽的詳細數據
    
* Function: getPlayerStatisticsFromSoup(soup)
    * Input: api 抓回來的 soup
    * Output: 把 df 存成 pickle
        * Data/season/start_date/event_key/team_full_name/player_stats.pickle
        * 例: Data/2017_2018/20170818/168428/Bayer 04 Leverkusen/player_stats.pickle
        
> Tony: 一次抓34天要頗久的 而且還只是球員數據那個api

In [ ]:
def getPlayerStatisticsFromSoup(soup):
    
    event_key = soup.find('event-metadata')['event-key']
    start_date = soup.find('event-metadata')['start-date-time'][:8]
    year = start_date[:4]
    month = start_date[4:6]
    # 判斷是今年 還是 前一年的比賽
    year = str(int(year)-1) if int(month) < 7 else year
    season = year + '_' + str(int(year)+1)
    
    for team in soup.find_all('team'):
        team_full_name = team.find('team-metadata').find('name')['full']

        # Variable initialization before each loop
        full_name, minutes_played, score = [],[],[]
        rating_value_goalie, rating_value_defenseman, rating_value_mid_fielder, rating_value_forward = [],[],[],[]
        duels_won, duels_won_ground, duels_won_header, duels_lost_ground, duels_lost_header, duels_lost, duels_won_percentage = [],[],[],[],[],[],[]
        passes_completed, passes_failed, passes_completions_percentage, passes_failed_percentage, passes, passes_short_total, balls_touched = [],[],[],[],[],[],[]
        tracking_distance, tracking_average_speed, tracking_max_speed, tracking_sprints, tracking_sprints_distance, tracking_fast_runs, tracking_fast_runs_distance, tracking_offensive_runs, tracking_offensive_runs_distance = [],[],[],[],[],[],[],[],[]
        dfl_distance, dfl_average_speed, dfl_max_speed = [],[],[]
        saves, catches_punches_crosses, catches_punches_corners, goals_against_total, penalty_saves, clear_cut_chance = [],[],[],[],[],[]
        shots_total, shots_on_goal_total, shots_off_post, offsides, corner_kicks, crosses, assists_total, shot_assists, freekicks, miss_chance, throw_in, punt, shots_penalty_shot_scored, shots_penalty_shot_missed, dfl_assists_total, shots_total_outside_box, shots_total_inside_box, shots_foot_inside_box, shots_foot_outside_box, shots_total_header = [],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[],[]
        fouls_commited, fouls_suffered, yellow_red_cards, red_cards, yellow_cards, penalty_caused = [],[],[],[],[],[]

        for player in team.find_all('player'):
            player_metadata = player.find('player-metadata')
            player_stats = player.find('player-stats')
            player_stats_soccer = player.find('player-stats-soccer')
            stats_soccer_defensive = player_stats_soccer.find('stats-soccer-defensive')
            stats_soccer_offensive = player_stats_soccer.find('stats-soccer-offensive')
            stats_soccer_foul = player_stats_soccer.find('stats-soccer-foul') 

            full_name.append(player_metadata.find('name')['full'])
            minutes_played.append(player_stats['minutes-played'])
            score.append(player_stats['score'])

            rating_value_goalie.append(player_stats.find('rating')['imp:rating-value-goalie'])
            rating_value_defenseman.append(player_stats.find('rating')['imp:rating-value-defenseman'])
            rating_value_mid_fielder.append(player_stats.find('rating')['imp:rating-value-mid-fielder'])
            rating_value_forward.append(player_stats.find('rating')['imp:rating-value-forward'])

            duels_won.append(player_stats_soccer['imp:duels-won'])
            duels_won_ground.append(player_stats_soccer['imp:duels-won-ground'])
            duels_won_header.append(player_stats_soccer['imp:duels-won-header'])
            duels_lost_ground.append(player_stats_soccer['imp:duels-lost-ground'])
            duels_lost_header.append(player_stats_soccer['imp:duels-lost-header'])
            duels_lost.append(player_stats_soccer['imp:duels-lost'])
            duels_won_percentage.append(player_stats_soccer['imp:duels-won-percentage'])

            passes_completed.append(player_stats_soccer['imp:passes-completed'])
            passes_failed.append(player_stats_soccer['imp:passes-failed'])
            passes_completions_percentage.append(player_stats_soccer['imp:passes-completions-percentage'])
            passes_failed_percentage.append(player_stats_soccer['imp:passes-failed-percentage'])
            passes.append(player_stats_soccer['imp:passes'])
            passes_short_total.append(player_stats_soccer['imp:passes-short-total'])
            balls_touched.append(player_stats_soccer['imp:balls-touched'])

            tracking_distance.append(player_stats_soccer['imp:tracking-distance'])
            tracking_average_speed.append(player_stats_soccer['imp:tracking-average-speed'])
            tracking_max_speed.append(player_stats_soccer['imp:tracking-max-speed'])
            tracking_sprints.append(player_stats_soccer['imp:tracking-sprints'])
            tracking_sprints_distance.append(player_stats_soccer['imp:tracking-sprints-distance'])
            tracking_fast_runs.append(player_stats_soccer['imp:tracking-fast-runs'])
            tracking_fast_runs_distance.append(player_stats_soccer['imp:tracking-fast-runs-distance'])
            tracking_offensive_runs.append(player_stats_soccer['imp:tracking-offensive-runs'])
            tracking_offensive_runs_distance.append(player_stats_soccer['imp:tracking-offensive-runs-distance'])

            dfl_distance.append(player_stats_soccer['dfl-distance'])
            dfl_average_speed.append(player_stats_soccer['dfl-average-speed'])
            dfl_max_speed.append(player_stats_soccer['dfl-max-speed'])

            saves.append(stats_soccer_defensive['saves'])
            catches_punches_crosses.append(stats_soccer_defensive['imp:catches-punches-crosses'])
            catches_punches_corners.append(stats_soccer_defensive['imp:catches-punches-corners'])
            goals_against_total.append(stats_soccer_defensive['goals-against-total'])
            penalty_saves.append(stats_soccer_defensive['imp:penalty-saves'])
            clear_cut_chance.append(stats_soccer_defensive['imp:clear-cut-chance'])

            shots_total.append(stats_soccer_offensive['shots-total'])
            shots_on_goal_total.append(stats_soccer_offensive['shots-on-goal-total'])
            shots_off_post.append(stats_soccer_offensive['imp:shots-off-post'])
            offsides.append(stats_soccer_offensive['offsides'])
            corner_kicks.append(stats_soccer_offensive['corner-kicks'])
            crosses.append(stats_soccer_offensive['imp:crosses'])
            assists_total.append(stats_soccer_offensive['assists-total'])
            shot_assists.append(stats_soccer_offensive['imp:shot-assists'])
            freekicks.append(stats_soccer_offensive['imp:freekicks'])
            miss_chance.append(stats_soccer_offensive['imp:miss-chance'])
            throw_in.append(stats_soccer_offensive['imp:throw-in'])
            punt.append(stats_soccer_offensive['imp:punt'])
            shots_penalty_shot_scored.append(stats_soccer_offensive['shots-penalty-shot-scored'])
            shots_penalty_shot_missed.append(stats_soccer_offensive['shots-penalty-shot-missed'])
            dfl_assists_total.append(stats_soccer_offensive['dfl-assists-total'])
            shots_total_outside_box.append(stats_soccer_offensive['imp:shots-total-outside-box'])
            shots_total_inside_box.append(stats_soccer_offensive['imp:shots-total-inside-box'])
            shots_foot_inside_box.append(stats_soccer_offensive['imp:shots-foot-inside-box'])
            shots_foot_outside_box.append(stats_soccer_offensive['imp:shots-foot-outside-box'])
            shots_total_header.append(stats_soccer_offensive['imp:shots-total-header'])

            fouls_commited.append(stats_soccer_foul['fouls-commited'])
            fouls_suffered.append(stats_soccer_foul['fouls-suffered'])
            yellow_red_cards.append(stats_soccer_foul['imp:yellow-red-cards'])
            red_cards.append(stats_soccer_foul['imp:red-cards'])
            yellow_cards.append(stats_soccer_foul['imp:yellow-cards'])
            penalty_caused.append(stats_soccer_foul['penalty-caused'])

        keys = ['full_name','minutes_played','score','rating_value_goalie','rating_value_defenseman','rating_value_mid_fielder','rating_value_forward','duels_won','duels_won_ground','duels_won_header','duels_lost_ground','duels_lost_header','duels_lost','duels_won_percentage','passes_completed','passes_failed','passes_completions_percentage','passes_failed_percentage','passes','passes_short_total','balls_touched','tracking_distance','tracking_average_speed','tracking_max_speed','tracking_sprints','tracking_sprints_distance','tracking_fast_runs','tracking_fast_runs_distance','tracking_offensive_runs','tracking_offensive_runs_distance','dfl_distance','dfl_average_speed','dfl_max_speed','saves','catches_punches_crosses','catches_punches_corners','goals_against_total','penalty_saves','clear_cut_chance','shots_total','shots_on_goal_total','shots_off_post','offsides','corner_kicks','crosses','assists_total','shot_assists','freekicks','miss_chance','throw_in','punt','shots_penalty_shot_scored','shots_penalty_shot_missed','dfl_assists_total','shots_total_outside_box','shots_total_inside_box','shots_foot_inside_box','shots_foot_outside_box','shots_total_header','fouls_commited','fouls_suffered','yellow_red_cards','red_cards','yellow_cards','penalty_caused']
        player_table_value = [full_name,minutes_played,score,rating_value_goalie,rating_value_defenseman,rating_value_mid_fielder,rating_value_forward,duels_won,duels_won_ground,duels_won_header,duels_lost_ground,duels_lost_header,duels_lost,duels_won_percentage,passes_completed,passes_failed,passes_completions_percentage,passes_failed_percentage,passes,passes_short_total,balls_touched,tracking_distance,tracking_average_speed,tracking_max_speed,tracking_sprints,tracking_sprints_distance,tracking_fast_runs,tracking_fast_runs_distance,tracking_offensive_runs,tracking_offensive_runs_distance,dfl_distance,dfl_average_speed,dfl_max_speed,saves,catches_punches_crosses,catches_punches_corners,goals_against_total,penalty_saves,clear_cut_chance,shots_total,shots_on_goal_total,shots_off_post,offsides,corner_kicks,crosses,assists_total,shot_assists,freekicks,miss_chance,throw_in,punt,shots_penalty_shot_scored,shots_penalty_shot_missed,dfl_assists_total,shots_total_outside_box,shots_total_inside_box,shots_foot_inside_box,shots_foot_outside_box,shots_total_header,fouls_commited,fouls_suffered,yellow_red_cards,red_cards,yellow_cards,penalty_caused]
        player_table = dict(zip(keys,player_table_value))

        player_df = pd.DataFrame(player_table)
        
        filename = 'Data/' + season +  '/' + start_date + '/' + event_key + '/' + team_full_name + '/' + 'player_stats.pickle'
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        with open(filename, 'wb') as handle:
            pickle.dump(df_matchday, handle, protocol=pickle.HIGHEST_PROTOCOL)

---

In [ ]:
# 抓matchday1~34 2017/18 抓完delay一會 避免被Ban
matchdays = {}
for i in range(34):
    api_url = 'https://www.bundesliga.com/fsSolrSearch/search?wt=json&rows=150&sort=kickoffTimeMS_l%20asc&q=indexName%3ASyncMatches%20AND%20competitionID_tlc%3ADFL-COM-000001%20AND%20seasonID_tlc%3A2017%2F2018%20AND%20matchday_l%3A'+str(i+1)
    url = session_requests.get(api_url, headers = headers) 
    matchdays[i] = json.loads(url.text)
    time.sleep(DELAY_SECONDS)
    if i % 5 == 0 and i != 0:
        print('Finished' + i + 'days')
print('All Finished!')

In [ ]:
# 抓 matchday 裡的 每一場比賽數據
for key,matchday in matchdays.items():
    print('Start to get matchday ' + str(key+1))
    
    for game in matchday['response']['docs']:   
        ID = game['dlProviderID_tlc']
        year = '2017'
        
        print('    Start to get game ' + ID)
        api_url = 'https://www.bundesliga.com/data/feed/51/' + year + '/player_stats/player_stats_' + ID + '.xml'
        url = session_requests.get(api_url, headers = headers) 
        soup = BeautifulSoup(url.content,'xml')
        time.sleep(DELAY_SECONDS)
        
        getPlayerStatisticsFromSoup(soup)
        
print('All Finished!')        

---

## TODO 
* 球隊相關數據 還沒抓